In [1]:
import math
import numpy as np
from sklearn.preprocessing import normalize

In [2]:
##init

# N = 2 #number of state
# M = 27
##create obv
obv = []
with open("/Users/chun/IdeaProjects/CS-271/output.txt", 'r') as file:
    for line in file:
        for char in line:
            if char == " ":
                obv.append(26)
            else:
                obv.append(ord(char.lower()) - ord('a'))
for i in obv:
    if i >= 27 or i < 0:
        print(i)

##initlizing a,b,pi
T = len(obv)
M = 27
N = 4
a = np.random.uniform(1/N - 1/(10*N),1/N + 1/(10*N),(N,N))
a = normalize(a, axis=1, norm='l1')
b = np.random.uniform(1/M - 1/(10*M),1/M + 1/(10*M),(N,M))
b = normalize(b, axis=1, norm='l1')
pi = np.random.uniform(1/N - 1/(10*N),1/N + 1/(10*N),(1,N))
pi = normalize(pi , axis=1, norm='l1')[0]

# a = np.matrix([[.7,.3],[.4,.6]])
# b = np.matrix([[.1,.4,.5], [.7,.2,.1]])
# pi = [.6,.4]
# obv = [0,1,0,2]

maxlters = 100
# a = np.zeros((N,N))
# b = np.zeros((N,M))
iters = 0
oldLogProb = -math.inf

In [3]:
##Caculating alpha_0:
# alpha = np.zeros((T,N))
# c = np.zeros(T)
# ## i is which state, so needs to go in front
# ## 0 is the number of obversation
# for i in range(0, N):
#     tmp = pi[i] * b[i,obv[0]]
#     alpha[0,i] = tmp
#     c[0] += alpha[0][i]
# c[0] = 1/c[0]
# for i in range(0,N):
#     alpha[0,i] *= c[0]
# print(alpha)

In [4]:
c = np.zeros(T)
alpha = np.zeros((T,N))
def alpha_t(a,b,obv,c,T,N):
#     print("alpha_t")
    ##Compute alpha_t(i)
##t from 1 to 2 to go over the last 2 obv
    ## i is which state, so needs to go in front
    ## 0 is the number of obversation
    for i in range(0, N):
        tmp = pi[i] * b[i,obv[0]]
        alpha[0,i] = tmp
        c[0] += alpha[0][i]
    c[0] = 1/c[0]
    for i in range(0,N):
        alpha[0,i] *= c[0]
    #print(alpha)
    for t in range(1, T):
        c[t] = 0
        for i in range(0, N):
            alpha[t,i] = 0
            ## i is state and t is obv, j is previous state
            for j in range(0,N):
                alpha[t,i] += alpha[t-1,j]*a[j,i]
            alpha[t,i] *= b[i,obv[t]]
            c[t] += alpha [t,i]
        c[t] = 1/ c[t]
        for i in range(0,N):
            alpha[t,i] *= c[t]
    return alpha
alpha = alpha_t(a=a,b=b,c=c,obv=obv,T=T,N=N)

In [5]:
# ##backward init
# beta = np.zeros((T,N))
# for row in beta:
#     row[-1] = 1
# for i in range(0, N):
#     beta[T-1,i] = c[T-1]

In [6]:
##beta-pass
def backward(a,b,c,T,N):
    beta = np.zeros((T,N))
    for row in beta:
        row[-1] = 1
    for i in range(0, N):
        beta[T-1,i] = c[T-1]
#     print("backward")
    for t in range(T-2,-1,-1):
        for i in range(0, N):
            beta[t,i] = 0
            for j in range(0, N):
                beta[t,i] += a[i,j] * b[j,obv[t+1]] * beta[t+1,j]
            beta[t,i] *= c[t]
    return beta
beta = backward(a=a,b=b,c=c,T=T,N=N)
beta

array([[26.47516795, 26.43447614, 26.44907551, 26.39673956],
       [27.98350266, 27.94376011, 27.96745742, 27.90158651],
       [26.2481893 , 26.32206582, 26.28371724, 26.34144942],
       ...,
       [27.44270861, 27.53158107, 27.48679876, 27.5254417 ],
       [27.5564882 , 27.46736845, 27.51393041, 27.45964147],
       [27.59857026, 27.59857026, 27.59857026, 27.59857026]])

In [7]:
# 

In [8]:
##Gammas
#a,b,alpha,beta,T,N
#a=a,b=b,alpha=alpha,beta=beta,T=T,N=N
def cal_gammas(a,b,alpha,beta,T,N):
    gammas = np.zeros((T,N))
    de_gammas = np.zeros((T,N,N))
    #print(T)
    for t in range(0, T-1):
        denom = 0
        for i in range(0, N):
            for j in range(0, N):
                denom += alpha[t,i] * a[i,j] * b[j,obv[t+1]] * beta[t+1,j]
        for i in range(0, N):
            gammas[t,i] = 0
            for j in range(0, N):
                de_gammas[t,i,j] = alpha[t,i] * a[i,j] * b[j,obv[t+1]]*beta[t+1,j] / denom
                gammas[t,i] += de_gammas[t,i,j]
    denom = 0
    for i in range(0, N):
        denom += alpha[T-1,i]
    for i in range(0, N):
        gammas[T-1,i] = alpha[T-1,i]/denom
    return gammas,de_gammas
gammas,de_gammas = cal_gammas(a=a,b=b,alpha=alpha,beta=beta,T=T,N=N)
# print(gammas)
print(gammas,de_gammas)

[[0.24665215 0.25176808 0.2505316  0.25104817]
 [0.25491113 0.24840708 0.23754672 0.25913507]
 [0.24158334 0.24745456 0.23908233 0.27187976]
 ...
 [0.23741953 0.26795853 0.24638778 0.24823416]
 [0.23861968 0.2675696  0.24569716 0.24811356]
 [0.25044418 0.23869262 0.24312382 0.26773938]] [[[0.06674307 0.05664785 0.05597341 0.06728781]
  [0.06300874 0.06701582 0.05730483 0.06443868]
  [0.06416766 0.06174823 0.05785626 0.06675945]
  [0.06099165 0.06299517 0.06641222 0.06064913]]

 [[0.06537197 0.0583679  0.05816816 0.0730031 ]
  [0.05891079 0.06591384 0.05684645 0.066736  ]
  [0.05763749 0.05834702 0.0551388  0.0664234 ]
  [0.05966309 0.0648258  0.06892892 0.06571726]]

 [[0.06257996 0.05920043 0.05783385 0.0619691 ]
  [0.05902745 0.069975   0.05915828 0.05929383]
  [0.05848339 0.06272681 0.05810826 0.05976387]
  [0.06282153 0.07231988 0.0753802  0.06135815]]

 ...

 [[0.06038363 0.05882994 0.05594832 0.06225764]
  [0.06273026 0.07658709 0.06303179 0.0656094 ]
  [0.05916011 0.06534904 0.0

In [9]:
print(a)
print("b")
print(b)
print("pi")
print(pi)
print("alpha")
print(alpha)
print("beta")
print(beta)
print("gammas")
print(gammas)
print("de_gammas")
print(de_gammas)

[[0.26181437 0.23377478 0.23488684 0.26952401]
 [0.24177108 0.27052524 0.23522545 0.25247823]
 [0.2475698  0.25062991 0.23879289 0.26300739]
 [0.23436734 0.25466009 0.27300104 0.23797153]]
b
[[0.03639584 0.04009726 0.04042316 0.03693826 0.03736394 0.04019297
  0.03468679 0.03698494 0.03465898 0.03907549 0.03352578 0.03628954
  0.03770068 0.0336451  0.03686916 0.03518141 0.03468611 0.03826781
  0.037347   0.03658746 0.03769668 0.03711799 0.03812033 0.03817982
  0.03949468 0.0357838  0.03668904]
 [0.03734643 0.03660381 0.03602477 0.0373549  0.03725721 0.03749852
  0.03923677 0.03520588 0.03511064 0.03818979 0.03732443 0.03960686
  0.03846662 0.03576628 0.03419185 0.03851191 0.03465101 0.03503223
  0.03824283 0.04018144 0.03915799 0.0337266  0.03520123 0.03582465
  0.03882123 0.03650696 0.03895714]
 [0.03516428 0.0382783  0.03491882 0.03898629 0.03700784 0.0345154
  0.03479413 0.0345927  0.03709434 0.03944387 0.04025468 0.03952856
  0.0391815  0.03836396 0.03589671 0.03639042 0.03485964 0

In [10]:
## re-estimate 
def reestimate(alpha,beta,gammas,de_gammas,obv,N,M,T):
#     print("reestimate")
    ##for pi
    for i in range(0, N):
        pi[i] = gammas[0,i]
    ##for A
    a = np.zeros((N,N))
    for i in range(0, N):
        for j in range(0,N):
            numer = 0
            denom = 0
            for t in range(0, T-1):
                numer += de_gammas[t,i,j]
                denom += gammas[t,i]
            a[i,j] = numer/denom
    ##for B
    b = np.zeros((N,M))
    for i in range(0,N):
        for j in range(0, M):
            numer = 0
            denom = 0
            for t in range(0, T):
                if obv[t] == j:
                    numer += gammas[t,i]
                denom += gammas[t,i]
            b[i,j] = numer/denom
    return a,b,pi
a,b,pi = reestimate(alpha=alpha,beta=beta,gammas=gammas,de_gammas=de_gammas,obv=obv,N=N,M=M,T=T)


In [11]:
print(a)
print("b")
print(b)
print("pi")
print(pi)
print("alpha")
print(alpha)
print("beta")
print(beta)
print("gammas")
print(gammas)
print("de_gammas")
print(de_gammas)

[[0.2601439  0.23560595 0.2334184  0.27083175]
 [0.24018868 0.27259025 0.23366518 0.2535559 ]
 [0.24594287 0.25260162 0.23727094 0.26418457]
 [0.23284671 0.25673137 0.27136363 0.23905829]]
b
[[0.06724411 0.01288549 0.03204648 0.03444527 0.10406845 0.01951948
  0.01368274 0.03881842 0.05870301 0.0018585  0.00405721 0.03563546
  0.01998278 0.05315084 0.06627941 0.01847765 0.00075046 0.05698562
  0.05761463 0.07641261 0.0221948  0.00893848 0.01242261 0.00225454
  0.01396075 0.00054617 0.16706402]
 [0.06805396 0.01159696 0.02814202 0.03440989 0.10228919 0.01794999
  0.01527487 0.03641768 0.05860987 0.00179221 0.00445004 0.03836968
  0.02009154 0.05576749 0.06057233 0.01992842 0.00074137 0.05144988
  0.05825596 0.08271838 0.02272255 0.00799623 0.01129928 0.00208933
  0.01355346 0.00054886 0.17490856]
 [0.06492194 0.01228051 0.02767062 0.03643569 0.10299608 0.01676255
  0.0137503  0.03626055 0.06277972 0.00187304 0.00486933 0.03883206
  0.02076229 0.06071839 0.06446808 0.01907963 0.00075581 

In [12]:
logProb = 0
for i in range(0, T):
    logProb += math.log(c[i])
logProb = -logProb
maxIters = 100
iters = 0
oldLogProb = -math.inf
while iters < maxIters and logProb > oldLogProb:
    iters += 1
    oldLogProb = logProb
    alpha = alpha_t(a=a,b=b,c=c,obv=obv,T=T,N=N)
    beta = backward(a=a,b=b,c=c,T=T,N=N)
    gammas, de_gammas = cal_gammas(a=a,b=b,alpha=alpha,beta=beta,T=T,N=N)
    a,b,pi = reestimate(alpha=alpha,beta=beta,gammas=gammas,de_gammas=de_gammas,obv=obv,N=N,M=M,T=T)
    for i in range(0, T):
        logProb += math.log(c[i])
    print(iters)
    print(a)
    print("b")
    print(b)
    print("pi")
    print(pi)
    

1
[[0.26014438 0.23556637 0.23340201 0.27088724]
 [0.24023119 0.27258551 0.23363896 0.25354435]
 [0.24593732 0.25261313 0.23726703 0.26418252]
 [0.2328098  0.25675706 0.27140822 0.23902492]]
b
[[0.06723361 0.01289216 0.0320673  0.03439967 0.10407356 0.01952056
  0.01367482 0.03885469 0.05872328 0.00185829 0.00405875 0.03562199
  0.01998991 0.05310309 0.06630233 0.01849517 0.00074874 0.05697602
  0.05756352 0.07644372 0.02219536 0.00894585 0.01243612 0.00225185
  0.01394423 0.00054656 0.16707884]
 [0.06805759 0.01160056 0.02814007 0.03442573 0.10225747 0.01794334
  0.01527896 0.03643119 0.05859995 0.00179308 0.00444771 0.03837342
  0.02008502 0.0557704  0.06055434 0.01993023 0.00074177 0.0514496
  0.05828519 0.08272174 0.0227118  0.00799015 0.01129895 0.00209016
  0.01355943 0.00054866 0.1749135 ]
 [0.06486561 0.01226636 0.0276767  0.03646943 0.10292166 0.01676409
  0.0137679  0.03624659 0.06275395 0.00186982 0.00486905 0.03883465
  0.02076461 0.06077986 0.06442747 0.01906483 0.00075569

7
[[0.26016471 0.23528886 0.23326406 0.27128237]
 [0.24050874 0.27256273 0.23345141 0.25347712]
 [0.24590681 0.25268847 0.2372305  0.26417422]
 [0.23254606 0.25692978 0.27175546 0.23876869]]
b
[[0.06716467 0.01293817 0.03220277 0.03412119 0.10408475 0.01952974
  0.01362655 0.03910269 0.0588424  0.00185777 0.00406878 0.03554061
  0.02003975 0.05279857 0.06644117 0.01860585 0.00073849 0.05692303
  0.05725397 0.07663052 0.02219303 0.0089944  0.01252396 0.00223544
  0.01384124 0.0005492  0.16715131]
 [0.06807064 0.01162114 0.02812873 0.03452123 0.10205298 0.0179005
  0.01530468 0.03650601 0.05853456 0.00179789 0.00443364 0.03839524
  0.02004445 0.05578222 0.0604309  0.01993993 0.00074396 0.05144289
  0.05845947 0.08277265 0.02264731 0.00795295 0.01129617 0.00209514
  0.01359455 0.00054733 0.17498285]
 [0.06450465 0.01218016 0.02771585 0.03668003 0.10243184 0.01677454
  0.01387417 0.03615515 0.06258185 0.00184999 0.00486746 0.03886216
  0.02078303 0.06117149 0.06416117 0.01897472 0.00075509

13
[[0.26021509 0.23493431 0.23304516 0.27180544]
 [0.24083329 0.27254756 0.2332035  0.25341565]
 [0.24588623 0.25277545 0.23716211 0.26417622]
 [0.23219552 0.25713962 0.27226953 0.23839534]]
b
[[0.0670842  0.01299592 0.03235887 0.03383125 0.10406038 0.01954325
  0.01357658 0.03941059 0.05895876 0.00185865 0.00408038 0.03545785
  0.02010304 0.05245657 0.06658206 0.01872744 0.00072829 0.05687701
  0.05693537 0.07682174 0.02218002 0.00905131 0.01262531 0.00221824
  0.01373006 0.00055242 0.16719443]
 [0.06806569 0.01163996 0.0281188  0.03461882 0.10181633 0.01785285
  0.01533308 0.03657042 0.05845679 0.00180205 0.00441932 0.03841587
  0.02000142 0.05578502 0.06027746 0.01994777 0.0007458  0.05142785
  0.05863406 0.08288278 0.02258191 0.00791445 0.01129219 0.0021001
  0.0136285  0.00054584 0.17512482]
 [0.06409689 0.01209094 0.02776066 0.03690814 0.10185434 0.01678784
  0.01398239 0.03604726 0.06237388 0.00182909 0.00486608 0.03891289
  0.02081031 0.06161238 0.06384934 0.01888167 0.0007547

19
[[0.26029653 0.23448644 0.23272096 0.27249606]
 [0.24121974 0.27253678 0.23287923 0.25336425]
 [0.24588457 0.25287549 0.23704383 0.26419611]
 [0.23173186 0.2573961  0.27300748 0.23786456]]
b
[[0.06698957 0.01306816 0.03254017 0.03352485 0.10399559 0.01956065
  0.01352432 0.03979316 0.05907451 0.00186113 0.004094   0.03537307
  0.02018241 0.05206548 0.06672382 0.01886249 0.000718   0.05683666
  0.05660123 0.07702496 0.02215446 0.00911833 0.01274361 0.00219975
  0.01360824 0.00055632 0.16720506]
 [0.06803756 0.01165709 0.0281117  0.03472062 0.10153446 0.01780075
  0.01536524 0.0366245  0.05836286 0.00180557 0.0044047  0.0384351
  0.01995606 0.05577979 0.06008816 0.01995395 0.00074731 0.05140509
  0.05881253 0.08306076 0.02251445 0.00787462 0.01128729 0.00210512
  0.01366116 0.00054419 0.17534938]
 [0.06362903 0.01199738 0.02781305 0.03715978 0.10116491 0.01680535
  0.01409385 0.03591681 0.06211913 0.0018068  0.0048649  0.03899182
  0.02084853 0.06212048 0.0634797  0.0187845  0.0007545

25
[[0.26041121 0.23392098 0.23225534 0.27341247]
 [0.24168886 0.27252599 0.23245507 0.25333009]
 [0.24591443 0.25299027 0.23684819 0.26424711]
 [0.23111714 0.25771279 0.27405447 0.2371156 ]]
b
[[0.06687646 0.01315898 0.03275323 0.03319582 0.10388223 0.01958143
  0.01346908 0.04027232 0.05919183 0.00186557 0.0041103  0.03528558
  0.02028169 0.05160937 0.06686381 0.01901455 0.00070747 0.0568004
  0.05624347 0.07725086 0.02211332 0.00919797 0.01288376 0.00217925
  0.01347254 0.00056105 0.16717763]
 [0.06797916 0.01167261 0.02810953 0.03482945 0.10118994 0.01774484
  0.01540272 0.03666765 0.0582478  0.00180844 0.00438972 0.03845292
  0.01990869 0.05576785 0.05985525 0.01995886 0.00074848 0.05137567
  0.05899955 0.08331431 0.02244366 0.00783341 0.01128186 0.00211029
  0.01369231 0.00054235 0.17567268]
 [0.06308223 0.01189769 0.02787572 0.03744323 0.10032887 0.01682901
  0.01421019 0.03575506 0.06180184 0.00178265 0.00486394 0.03910606
  0.02090062 0.06272102 0.06303463 0.01868167 0.0007548

31
[[0.26056234 0.23320119 0.23159311 0.27464336]
 [0.24227062 0.2725084  0.23189547 0.25332551]
 [0.24599434 0.25312132 0.2365321  0.26435224]
 [0.23029522 0.25810943 0.27554138 0.23605397]]
b
[[0.06673794 0.01327459 0.03300769 0.03283613 0.10370683 0.01960501
  0.01341018 0.04088145 0.05931305 0.00187252 0.00413031 0.03519464
  0.02040662 0.05106533 0.06699677 0.01918872 0.00069653 0.05676617
  0.05585144 0.07751506 0.02205183 0.00929394 0.01305299 0.00215572
  0.01331855 0.00056683 0.16710315]
 [0.06788002 0.01168665 0.02811556 0.03494938 0.10075784 0.01768622
  0.01544782 0.03669805 0.05810464 0.00181067 0.0043744  0.03846978
  0.01986007 0.05575118 0.05956785 0.01996326 0.00074934 0.05134158
  0.05920163 0.08365101 0.0223682  0.0077908  0.01127657 0.00211571
  0.01372143 0.00054031 0.17612001]
 [0.06242869 0.0117894  0.02795262 0.03777047 0.09929446 0.01686179
  0.01433355 0.03554917 0.06139843 0.00175604 0.00486317 0.03926621
  0.02097092 0.06345139 0.06248794 0.01857109 0.000755

37
[[0.26075385 0.23226961 0.23064628 0.27633026]
 [0.24301004 0.27247235 0.23114484 0.25337278]
 [0.24615288 0.25326893 0.2360253  0.26455289]
 [0.22917981 0.25861576 0.27767783 0.2345266 ]]
b
[[0.06656238 0.01342489 0.03331848 0.03243481 0.10344656 0.01963071
  0.01334693 0.04167373 0.05944062 0.00188287 0.00415573 0.0350994
  0.02056618 0.05039881 0.0671125  0.01939265 0.00068493 0.05673089
  0.05541041 0.07784117 0.02196223 0.00941192 0.01326253 0.00212753
  0.0131399  0.00057402 0.16696722]
 [0.06772358 0.01169942 0.02813502 0.03508649 0.10019976 0.01762683
  0.01550413 0.03671183 0.05792259 0.00181223 0.00435886 0.03848695
  0.01981168 0.05573281 0.05920944 0.01996858 0.00074991 0.05130644
  0.05942829 0.08408135 0.02228641 0.00774684 0.01127247 0.00212151
  0.01374755 0.00053804 0.17673096]
 [0.06162519 0.01166882 0.02805001 0.03815973 0.09798025 0.01690852
  0.01446691 0.03527951 0.06087183 0.00172602 0.00486252 0.03948906
  0.02106628 0.06437045 0.0617987  0.01844979 0.0007572

43
[[0.26098897 0.23103132 0.22926632 0.27871339]
 [0.24397832 0.27239596 0.23011137 0.25351435]
 [0.24643656 0.25342951 0.23520666 0.26492728]
 [0.22763128 0.25927817 0.28081909 0.23227146]]
b
[[0.06632897 0.01362651 0.03370996 0.03197627 0.10306083 0.01965776
  0.01327892 0.04273881 0.05957671 0.00189821 0.00418951 0.03499903
  0.02077524 0.04955461 0.06719081 0.01963859 0.00067231 0.05668964
  0.05489874 0.07826699 0.02183133 0.00956105 0.01353088 0.00209195
  0.01292697 0.00058317 0.16674622]
 [0.06748151 0.01171135 0.02817653 0.03525039 0.09945149 0.01757022
  0.01557737 0.03670134 0.05768358 0.0018131  0.00434345 0.03850741
  0.01976638 0.05571762 0.05875286 0.01997745 0.00075023 0.05127699
  0.05969442 0.08462566 0.02219609 0.00770155 0.01127138 0.00212776
  0.01376871 0.00053545 0.17756972]
 [0.06060025 0.01153031 0.02817859 0.03864065 0.09624962 0.01697766
  0.01461448 0.03491405 0.06015986 0.00169114 0.00486183 0.03980298
  0.02119839 0.0655779  0.06089885 0.01831339 0.000759

49
[[0.26126498 0.22931634 0.22718209 0.28223659]
 [0.24529628 0.27223441 0.2286317  0.25383761]
 [0.24692594 0.25358772 0.23385102 0.26563531]
 [0.22540671 0.26017172 0.28561616 0.22880541]]
b
[[0.06599653 0.01390972 0.03422506 0.03143733 0.10247103 0.01968567
  0.01320692 0.04424068 0.05972165 0.00192146 0.00423735 0.03489319
  0.02106048 0.04843782 0.06718895 0.01994774 0.00065818 0.05663369
  0.05428242 0.07885839 0.02163518 0.00975712 0.01389099 0.00204395
  0.0126642  0.00059521 0.16639906]
 [0.06710114 0.01172329 0.02825547 0.03545767 0.09839543 0.01752334
  0.01567742 0.03665145 0.0573546  0.00181321 0.00432897 0.03853803
  0.0197298  0.05571422 0.058149   0.01999496 0.0007504  0.0512666
  0.06002592 0.0853286  0.02209394 0.00765472 0.01127656 0.00213435
  0.01378092 0.00053243 0.17874756]
 [0.05922543 0.01136461 0.02835838 0.03926555 0.09385372 0.01708558
  0.01478261 0.03439591 0.05914911 0.00164899 0.00486076 0.0402603
  0.02138907 0.06725856 0.05966538 0.01815512 0.00076441

KeyboardInterrupt: 

In [ ]:
print("a")
print(a)
print("b")
print(b)
print("pi")
print(pi)

In [ ]:
b

In [ ]:
pi